In [14]:
# %pip install openpyxl
# %pip install scikit-learn


  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [1]:
#import the relevant libraries
import os
import glob

import numpy as np
import pandas as pd
import openpyxl as op
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:
#load our data
df = pd.read_excel('../../../esundhed/esundhed_df.xlsx')

In [46]:
# set first row as column names
# df.columns = df.iloc[0]

# delete first row
# df = df[1:]

# delete column 'variable'
# df = df.drop(columns='Variabel')

In [47]:
# check first 10 rows
pd.set_option('display.max_columns', None)
df.head(10)

,ATC og navn,Enhed,Kvartal,2. Kvartal,1. Kvartal
1,A01A Stomatologica,Antal,NaN,*,*
2,A01AA Midler mod caries,Antal,NaN,26.987,29.372
3,A01AA01 Natriumfluorid,Antal,NaN,26.987,29.372
4,A01AB Antiinfectiva til lokal brug i mundhulen,Antal,NaN,7.726,7.605
5,A01AB09 Miconazol,Antal,NaN,7.726,7.605
6,A01AD Andre midler til lokal behandling i mund...,Antal,NaN,*,*
7,A01AD02 Benzydamin,Antal,NaN,0,0
8,A01AD11 Diverse,Antal,NaN,-,0
9,A02A Antacida,Antal,NaN,*,*
10,A02AA Magnesiumforbindelser,Antal,NaN,*,*


In [48]:
df = df.drop(columns='Kvartal')

In [49]:
df.head(10)

,ATC og navn,Enhed,2. Kvartal,1. Kvartal
1,A01A Stomatologica,Antal,*,*
2,A01AA Midler mod caries,Antal,26.987,29.372
3,A01AA01 Natriumfluorid,Antal,26.987,29.372
4,A01AB Antiinfectiva til lokal brug i mundhulen,Antal,7.726,7.605
5,A01AB09 Miconazol,Antal,7.726,7.605
6,A01AD Andre midler til lokal behandling i mund...,Antal,*,*
7,A01AD02 Benzydamin,Antal,0,0
8,A01AD11 Diverse,Antal,-,0
9,A02A Antacida,Antal,*,*
10,A02AA Magnesiumforbindelser,Antal,*,*


In [50]:
# show all distinct values in the column 'Enhed'
df['Enhed'].unique()

array(['Antal'], dtype=object)

In [51]:
df = df.drop(columns='Enhed')

In [52]:
df.head(10)

,ATC og navn,2. Kvartal,1. Kvartal
1,A01A Stomatologica,*,*
2,A01AA Midler mod caries,26.987,29.372
3,A01AA01 Natriumfluorid,26.987,29.372
4,A01AB Antiinfectiva til lokal brug i mundhulen,7.726,7.605
5,A01AB09 Miconazol,7.726,7.605
6,A01AD Andre midler til lokal behandling i mund...,*,*
7,A01AD02 Benzydamin,0,0
8,A01AD11 Diverse,-,0
9,A02A Antacida,*,*
10,A02AA Magnesiumforbindelser,*,*


In [55]:
# drop all values with '*'
df = df[~df['1. Kvartal'].str.contains('\*')]

<>:2: SyntaxWarning: invalid escape sequence '\*'
<>:2: SyntaxWarning: invalid escape sequence '\*'
C:\Users\Josef\AppData\Local\Temp\ipykernel_27808\838017485.py:2: SyntaxWarning: invalid escape sequence '\*'
  df = df[~df['1. Kvartal'].str.contains('\*')]


In [57]:
df

,ATC og navn,2. Kvartal,1. Kvartal
2,A01AA Midler mod caries,26.987,29.372
3,A01AA01 Natriumfluorid,26.987,29.372
4,A01AB Antiinfectiva til lokal brug i mundhulen,7.726,7.605
5,A01AB09 Miconazol,7.726,7.605
7,A01AD02 Benzydamin,0,0
...,...,...,...
1602,V09X* n/a,<5,-
1603,V09X*** (Ufuldstændig ATC-kode),<5,-
1604,V10X Andre radiofarmaka til terapeutisk brug,<5,<5
1605,V10X* n/a,<5,<5


In [59]:
df = df[~df['1. Kvartal'].str.contains('-')]

In [60]:
df

,ATC og navn,2. Kvartal,1. Kvartal
2,A01AA Midler mod caries,26.987,29.372
3,A01AA01 Natriumfluorid,26.987,29.372
4,A01AB Antiinfectiva til lokal brug i mundhulen,7.726,7.605
5,A01AB09 Miconazol,7.726,7.605
7,A01AD02 Benzydamin,0,0
...,...,...,...
1599,V08CA04 Gadoteridol,0,0
1600,V08CA09 Gadobutrol,0,0
1604,V10X Andre radiofarmaka til terapeutisk brug,<5,<5
1605,V10X* n/a,<5,<5


In [66]:
# remove the '.' in the numbers. For instance, '1.000' will be '1000'
df['1. Kvartal'] = df['1. Kvartal'].str.replace('.', '')

In [67]:
df

,ATC og navn,2. Kvartal,1. Kvartal
2,A01AA Midler mod caries,26.987,29372
3,A01AA01 Natriumfluorid,26.987,29372
4,A01AB Antiinfectiva til lokal brug i mundhulen,7.726,7605
5,A01AB09 Miconazol,7.726,7605
7,A01AD02 Benzydamin,0,0
...,...,...,...
1599,V08CA04 Gadoteridol,0,0
1600,V08CA09 Gadobutrol,0,0
1604,V10X Andre radiofarmaka til terapeutisk brug,<5,<5
1605,V10X* n/a,<5,<5


In [68]:
# show all the distinct non number values in the column '1. Kvartal'
df[~df['1. Kvartal'].str.isnumeric()]['1. Kvartal'].unique()

array(['<5'], dtype=object)

In [69]:
# do the same cleaning we did for '1. Kvartal' for the other column '2. Kvartal'
df = df[~df['2. Kvartal'].str.contains('\*')]
df = df[~df['2. Kvartal'].str.contains('-')]
df['2. Kvartal'] = df['2. Kvartal'].str.replace('.', '')

<>:2: SyntaxWarning: invalid escape sequence '\*'
<>:2: SyntaxWarning: invalid escape sequence '\*'
C:\Users\Josef\AppData\Local\Temp\ipykernel_27808\3342028008.py:2: SyntaxWarning: invalid escape sequence '\*'
  df = df[~df['2. Kvartal'].str.contains('\*')]


In [73]:
# show all the distinct non number values in the column '1. Kvartal'
df[~df['2. Kvartal'].str.isnumeric()]['1. Kvartal'].unique()

array(['<5', '16', '5', '6', '0', '7', '9', '18', '54', '53', '235'],
      dtype=object)

In [77]:
df

,ATC og navn,2. Kvartal,1. Kvartal
2,A01AA Midler mod caries,26987,29372
3,A01AA01 Natriumfluorid,26987,29372
4,A01AB Antiinfectiva til lokal brug i mundhulen,7726,7605
5,A01AB09 Miconazol,7726,7605
7,A01AD02 Benzydamin,0,0
...,...,...,...
1596,V08CA Paramagnetiske kontrastmidler,0,0
1597,V08CA01 Gadopentetinsyre,0,0
1598,V08CA02 Gadoterinsyre,0,0
1599,V08CA04 Gadoteridol,0,0


In [78]:
# fix the indexing of the dataframe
df = df.reset_index(drop=True)

In [79]:
df

,ATC og navn,2. Kvartal,1. Kvartal
0,A01AA Midler mod caries,26987,29372
1,A01AA01 Natriumfluorid,26987,29372
2,A01AB Antiinfectiva til lokal brug i mundhulen,7726,7605
3,A01AB09 Miconazol,7726,7605
4,A01AD02 Benzydamin,0,0
...,...,...,...
1250,V08CA Paramagnetiske kontrastmidler,0,0
1251,V08CA01 Gadopentetinsyre,0,0
1252,V08CA02 Gadoterinsyre,0,0
1253,V08CA04 Gadoteridol,0,0


In [83]:
# split atc and name into two separate columns in the 'ATC og navn' column. Split them on the first space character
df[['WHO ATC 5 Code', 'Name in DA']] = df['ATC og navn'].str.split(n=1, expand=True)

df = df.drop(columns='ATC og navn')

In [85]:
# change the order of the columns. First should be the ATC code, then the name, then the two quarters
df = df[['WHO ATC 5 Code', 'Name in DA', '1. Kvartal', '2. Kvartal']]

In [86]:
df

,WHO ATC 5 Code,Name in DA,1. Kvartal,2. Kvartal
0,A01AA,Midler mod caries,29372,26987
1,A01AA01,Natriumfluorid,29372,26987
2,A01AB,Antiinfectiva til lokal brug i mundhulen,7605,7726
3,A01AB09,Miconazol,7605,7726
4,A01AD02,Benzydamin,0,0
...,...,...,...,...
1250,V08CA,Paramagnetiske kontrastmidler,0,0
1251,V08CA01,Gadopentetinsyre,0,0
1252,V08CA02,Gadoterinsyre,0,0
1253,V08CA04,Gadoteridol,0,0


In [87]:
# save and export this to a csv file
df.to_csv('esundhed_df_cleaned.csv', index=False)